In [2]:
import os
import pandas as pd

from itertools import combinations
from collections import Counter

folder_path = '../../Datasets/Sales'
dataset_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

In [3]:
merged_data = pd.DataFrame()

for file in dataset_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    merged_data = pd.concat([merged_data, df], ignore_index=True)

merged_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,259358,34in Ultrawide Monitor,1,379.99,10/28/19 10:56,"609 Cherry St, Dallas, TX 75001"
1,259359,27in 4K Gaming Monitor,1,389.99,10/28/19 17:26,"225 5th St, Los Angeles, CA 90001"
2,259360,AAA Batteries (4-pack),2,2.99,10/24/19 17:20,"967 12th St, New York City, NY 10001"
3,259361,27in FHD Monitor,1,149.99,10/14/19 22:26,"628 Jefferson St, New York City, NY 10001"
4,259362,Wired Headphones,1,11.99,10/07/19 16:10,"534 14th St, Los Angeles, CA 90001"


### Adding a Month column to see seasonality and check the best month for sales

In [4]:
merged_data = merged_data[merged_data['Order Date'] != 'Order Date']
merged_data['Order Date'] = pd.to_datetime(merged_data['Order Date'], format='%m/%d/%y %H:%M')

In [5]:
if merged_data['Order Date'].isna().any():
    all_columns_nan = merged_data[merged_data['Order Date'].isna()].isna().all(axis=1)
    merged_data = merged_data[~(merged_data['Order Date'].isna() & all_columns_nan)]

merged_data['Order Date'] = merged_data['Order Date'].fillna(method='ffill').fillna(method='bfill')
#merged_data['Month'] = merged_data['Order Date'].str[0:2].astype('int16')
merged_data['Month'] = merged_data['Order Date'].dt.strftime('%B')

### Adding a Sales column

In [6]:
merged_data['Quantity Ordered'] = merged_data['Quantity Ordered'].astype('int16')
merged_data['Price Each'] = pd.to_numeric(merged_data['Price Each'])
merged_data['Sale Value'] = merged_data['Quantity Ordered'] * merged_data['Price Each']

### Adding a City column and Hour of sale column

In [7]:
merged_data['City'] = merged_data['Purchase Address'].apply(lambda x: x.split(',')[1] + ' ' + x.split(',')[2].split(' ')[1])
merged_data['Hour of Sale'] = merged_data['Order Date'].dt.strftime('%H:%M')

### SAVING PROCESSED DATASET

### Grouping most sold products

In [9]:
products = merged_data[merged_data['Order ID'].duplicated(keep=False)]
products.loc[:, 'Grouped'] = products.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))
products = products[['Order ID', 'Grouped']].drop_duplicates()
products.head()

/tmp/ipykernel_260637/2688769586.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products.loc[:, 'Grouped'] = products.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))


,Order ID,Grouped
62,259420,"Google Phone,USB-C Charging Cable"
65,259422,"Google Phone,USB-C Charging Cable"
82,259438,"iPhone,Lightning Charging Cable"
103,259458,"34in Ultrawide Monitor,ThinkPad Laptop"
123,259477,"LG Dryer,AAA Batteries (4-pack)"


In [10]:
count = Counter()

for _ in products['Grouped']:
    row = _.split(',')
    count.update(Counter(combinations(row, 2)))

for k, v in count.most_common(10):
    print(f'{k}: {v}')

('iPhone', 'Lightning Charging Cable'): 1005
('Google Phone', 'USB-C Charging Cable'): 987
('iPhone', 'Wired Headphones'): 447
('Google Phone', 'Wired Headphones'): 414
('Vareebadd Phone', 'USB-C Charging Cable'): 361
('iPhone', 'Apple Airpods Headphones'): 360
('Google Phone', 'Bose SoundSport Headphones'): 220
('USB-C Charging Cable', 'Wired Headphones'): 160
('Vareebadd Phone', 'Wired Headphones'): 143
('Lightning Charging Cable', 'Wired Headphones'): 92


In [11]:
prods = merged_data.groupby('Product')
qtd_ord = prods['Quantity Ordered'].sum()
qtd_ord.idxmax()

'AAA Batteries (4-pack)'

In [12]:
merged_data.groupby('City')['Sale Value'].sum()

City
 Atlanta GA          2795498.58
 Austin TX           1819581.75
 Boston MA           3661642.01
 Dallas TX           2767975.40
 Los Angeles CA      5452570.80
 New York City NY    4664317.43
 Portland ME          449758.27
 Portland OR         1870732.34
 San Francisco CA    8262203.91
 Seattle WA          2747755.48
Name: Sale Value, dtype: float64